# 10 Minionology: HVSR with seismic nodes/stations
### Skience2025 practical on HVSR, node installation, applications, Geopsy, continuous data analysis

##### Authors:
* Koen Van Noten ([@KoenVanNoten](https://github.com/KoenVanNoten))
* Thomas Lecocq ([@seismotom](https://github.com/ThomasLecocq))

##### Introduction:
Three-component __seismic nodes__ are conquering the world these days as lightweight smart seismic sensors. This notebook provides a guideline how to handle seismic node data and perform H/V spectral ratio analysis of ambient noise (mHVSR of HVSRN) recorded with seismic nodes. We'll show you some methods:
* how to perform mHVSR using the Geopsy software manually
* how to do the same exercise automatically 
* and we'll open the discussion towards an automatic solution. 

SmartSolo 3D Nodes (https://smartsolo.com/cp.php?id=3) are easy to deploy, have long battery life (2-4 weeks), are modular to easily replace battery and are fastly charged. The picture below shows the modular design of the IGU-16HR 3C series where 3C sensors are installed on a standard battery pack (usually used with the 1C nodes). The tripod feet allow them to be used in urban settings. As they resemble to the Minions, we batised _data analysis with nodes_ as __Minionology__. We further refer to Zeckra et al. (preprint) to a technical introduction on the performance of the IGU-16HR 3C series and to Van Noten et al. (2022, 2020) for Virtual Borehole information and codes.

##### References:
* Van Noten, K., Lecocq, T., Goffin, C., Meyvis, B., Molron, J., Debacker, T.N. & Devleeschouwer, X. 2022. Brussels’ bedrock paleorelief from borehole-controlled powerlaws linking polarised H/V resonance frequencies and sediment thickness. _Journal of Seismology_ 26, 35-55. DOI: https://doi.org/10.1007/s10950-021-10039-8 pdf: https://publi2-as.oma.be/record/5626/files/2022_VanNotenetal_HVSR_Powerlaw_Brussels.pdf 
* Van Noten, K, Lecocq, Buddha Power, B. (2020). HVSR to Virtual Borehole (1.0). Zenodo. https://doi.org/10.5281/zenodo.4276310
* Zeckra, M., Van Noten, K., Lecocq, T. Preprint. Sensitivity, Accuracy and Limits of the Lightweight Three-Component SmartSolo Geophone Sensor (5 Hz) for Seismological Applications. Preprint on: https://doi.org/10.31223/X5F073

<img src="Minions Seismology.be.jpg" width=600></img>

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
import os
import glob
from obspy import read
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
from matplotlib.dates import DateFormatter, DayLocator, HourLocator
from matplotlib.ticker import MaxNLocator, StrMethodFormatter, NullFormatter
import pytz
import timeit
import matplotlib.ticker as ticker

# use ipython notebook in a wider screen
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

# run the HVSR Minionology function notebook
%run 00_HVSR_Minionology_definitions.ipynb

In [ ]:
# Path to where you have copied the DATA/ folder (that contains the SDS, RESP etc folders)
DATA_PATH = "DATA"

## Step 1. Manual process of ambient noise data with Geopsy
Before processing any data I will showcase a manual data analysis in Geopsy. Credits to Marc Wathelet & team (ISTerre Grenoble) for years of Geopsy coding, development and improvement. 

We will first manually process the seismic data in Geopsy using the _HV module_ and following standard processing steps. In the Geopsy H/V module use following parameters (also explained here: https://www.geopsy.org/wiki/index.php/H/V_spectral_ratio). For a deep basin we need e.g.
* __Length__: 360s
* __Overlap__: 50%
* __Relative treshhold__: 40%
* __Taper__: 5% Tukey
* __Konno-Omachi smoothing__: log 20%
* __Squared Average__
* __Output__: 0.10Hz - 100 Hz
* __Step Count__: 2000

After computation, manually clean the H/V curve by _Reject time windows_ (right click on the graph) and select those curves that deviate from the mean curve. Then recompute the H/V curve by pressing the black arrow next to _select_ -> _clear_ -> _Remove grayed_ and press _Start_ again.

To save the .HV results do a _Tools_ -> _Save Results_ and save it in the  __Manual_Analysed__ folder

## Step 2. Auto_process ambient noise data with Geopsy module

In this part we will Auto_process mseed data of 4 sensors using the Geopsy command line.
* First select the HVSR params to process the data. This is similar as the manual processing in Geopsy

In [ ]:
##### Give location of geopsy
# geopsy_exe = "geopsy-hv" """ FOR Linux
geopsy_exe = 'C:/Users/koenvn/geopsypack-win64-3.4.2/bin/geopsy-hv.exe' # For Windows - locate the Geopsypack.win64-3.4.2 folder

* Run the command line below and define the options. Results will be stores in the Auto_process folder

#### HV PARAMS
* process_len = 3600. # overall processing lengths[s] standard 1 hour - (time for which we compute the total HV-curve)
* time_window_len = 360. # window length for each HV curve
* win_overlap = 50 # % overlapping windows
* threshold_pct = 0.4 # percentage of treshhold to acccept
* KO = 0.2 # Konno_Omachi Smoothing (in digits)
* min_freq = 0.1 # lower frequency bound [Hz]
* max_freq = 100 # upper frequency bound# upper frequency bound [Hz]
* N_samples = 2000 # Nr of SAMPLES_NUMBER_FREQUENCY={S_N_FREQ}

#### POLARISATION PARAMS
* azimuth = 0 ##HORIZONTAL_AZIMUTH is used only when HORIZONTAL_COMPONENTS== 'Azimuth'
* want_rotation = False # # in case you want run the HV rotate module
* rotation_steps = 10 # [degree]

In [ ]:
%run Auto_process_HV.py -h

In [ ]:
%run Auto_process_HV.py -p 3600 -w 360 -o 50 -t 0.99 -k 0.2 -l 0.1 -u 100 -n 2000

## Step 3. Grab the processed .hv data in the Analysed_Skience folder.

This function reads all .hv data and saves the information in a csv database file as *NODE-NAME*_HVSR_all_mindate_maxdate

* first try your own processing in the Analysed_Skience25 folder

In [ ]:
### Get the HV files from the continuous analysis
## Give folder where to read the Project HVSR data, later we read it from the HVSR database
base_folder = os.getcwd()

# Give folder where processed HV data are stored
data_folder = r'Analysed_Skience25' #data computed during SKIENCE
os.makedirs(data_folder, exist_ok=True)

# Give project name (can be whatever) and time zone
Project = 'SKIENCE2025'
time_zone = 'Europe/Paris'

## Want to find the f0 throughout the whole processed HVSR curve? Then freq_lim = False
## If True, give frequency limits between you want to find the f0
freq_lim = False
ylim_min , ylim_max = 0.1, 0.4

## The SmartSolonodes also provide temperature data. 
## If you want to plot these later on top of HVSR data, give the project temperature log and store it in the project folder.
temperature = False
temp_log = 'tbl_temperature_log.csv'
    
###################################################################
# MAIN
###################################################################
temp_log = os.path.join(base_folder, temp_log)
print(data_folder)
grab_continuous_HV()

## Step 4. Explore the continuous HVSR from one day

The code in below explores the HVSR results. It will read the hourly .HV processed data and plots it with time. You can zoom in into the frequency band of interest and play with the start- and enddate. 

* first plot your own analysis of 30 January 2024


* **Question 1: how many time you need to find the resonance of a building: hours? days? months?**

* **Question 2: how many time you need to find the resonance frequency of the soil: hours? days? months?**

* **Question 3: Can you see the resonance frequency of the soil at the rooftop in HB01, HB02, HB03 ?**

* **Question 4: Can you see the resonance of the building on the basefloor in HB04 ?**


In [ ]:
# Give project name and time zone
Project = 'SKIENCE2025' #name it into whatever you want
time_zone = 'Europe/Paris'
base_folder = os.getcwd()
data_folder = r'Analysed_Skience25' # this is the folder for the test results of 20230501

###### activating the folder below browser through 2 yr of data
#data_folder = r'Analysed'

#Plot between dates Format %YYYY-MM-DD hh:00:00 
start_date = '2024-01-30 00:00:00'
end_date = '2024-02-01 23:00:00'

# Want to plot local or UTC time?
time = 'Local_Time'
#time = 'UTC'

# want to zoom on the certain frequency and amplitude range?
zoom = True
#### frequency limit
ylim_min , ylim_max = 0.1, 100
### amplitude limits
xlim_min, xlim_max = 0, 10

## Wan't the amplitudes in logscale ?
log_xscale = False

# the figure will be saved, don't plot it as well (slows down the process)
plot = False

# thickness of the HV line
HV_line_thickness = 10

################
# MAIN
################
plot_continuous_HV()

print('job done')

## STep 5. now explore one month of HV data (January 2024)

In [ ]:
# Give project name and time zone
Project = 'SKIENCE2025' #name it into whatever you want
time_zone = 'Europe/Paris'
base_folder = os.getcwd()
data_folder = os.path.join(DATA_PATH, 'Analysed_Server') # this is the folder for the test results of 20230501

#Plot between dates Format %YYYY-MM-DD hh:00:00 
start_date = '2024-01-01 01:00:00'
end_date = '2024-01-02 23:00:00'

# Want to plot local or UTC time?
time = 'Local_Time'
#time = 'UTC'

# want to zoom on the certain frequency and amplitude range?
zoom = True
#### frequency limit
ylim_min , ylim_max = 0.1, 100
### amplitude limites
xlim_min, xlim_max = 0, 30

## Wan't the amplitudes in logscale ?
log_xscale = False

# the figure will be saved, don't plot it as well (slows down the process)
plot = False

# thickness of the HV line
HV_line_thickness = 5

################
# MAIN
################
plot_continuous_HV()

print('job done')

## Step 5: Plot a Virtual Borehole from the 2 year mean HVSR curve of HB04

You can use a fixed Vs in center of Grenoble: 600 - 750 m/s

or if you find a Powerlaw relation applicable for deep basins/Grenoble basin, change the a and b values

In [ ]:
Project = 'SKIENCE2025'
HVSR_Project = 'Woodbuilding Grenoble'
base_folder = r''

# Give folder where processed HV data are stored
data_folder = os.path.join(DATA_PATH, 'Analysed_Server')
# data_folder = 'Analysed'
ID = 'HB04'
Z = 216  #surface height

# Choose if you want to use the Geopsy exported values or want to interpolate between 0 and 15000 frequency values
# See annotations in "get_interpolated_values_from_HV for details
interpolate = False

# Choose if the amplitude on the frequency-amplitude plot needs to be selected automatically or manually
auto_amplitude = True
manual_amplitude = 10

# Choose between which frequencies you want to plot. Default = between 0.5 Hz and 50 Hz
freq = [0.1, 100]

## f0 needs to be converted to depth by: 

## 1. by using a Powerlaw relation between resonance frequency and depth according to the formula: depth = a * power(f0, b)
## a & b values of the Regional powerlaw relation (R') of Van Noten et al. 2022. (applicable for Brussels)

#depth_conversion = 'powerlaw'
a_pw = 88.631     # a value
b_pw = -1.683    # b value

## or by using a fixed velocity
depth_conversion = 'Vs'
Vs = 700 # m/s


###################################################################
HV_file = os.path.join(data_folder, '8N_%s_HVSR_mean.hv'%ID)
print(HV_file)
HV_to_virtual_borehole(HV_file, ID, Z, 'f0_win', Project)
#save it by node name
savefile = os.path.join(data_folder, '%s_Virtual_Borehole.png'%ID)
plt.savefig(savefile, format= 'png', dpi = 300)

## Step 6: Make a polar plot from the geopsy HV rotate module output .grid file 
Go back to Geopsy and activate the HV rotate module. Save the results.

The Geopsy output is not intuitive as polar data are plotted in an __X__ (Frequency) - __Y__ (Azimuth) diagram in stead of a 360° diagram. The script in below loads one or all Geopsy HV rotate module .grid files and replots it into a more understandable polar plot. It will search the azimuth at which the maximum resonance frequency occurs. 

In below, first several options are given to plot the HV polar plot, then the polar plot is made.
Following data is returned:
* __A_max__: maximum amplitude at resonance frequency deduced from the HVSR polarisation analysis 
* __max_freq__: Resonance frequency at A_max
* __max_Azi__: Azimuth at which resonance frequency is maximum (deduced from polarisation analysis)
* __A_min__: minimum amplitude at resonance frequency deduced from the HVSR polarisation analysis 
* __min_freq__:  Azimuth at which resonance frequency is minimal (deduced from polarisation analysis)

In [ ]:
plot_polarisation_data?

In [ ]:
#€ give HV.grid file
data_folder = os.path.join(DATA_PATH, 'Analysed_Server')
# data_folder = 'Analysed'
ID = 'HB04'

# If freq_range = True, search for the maximum azimuthal values in a certain frequency range 
# If freq_range = False, code will take the max range in the .grid file
freq_range = False
f_range = [1.15, 1.4]

# Wanna plot the polarisation figure?
plot_fig = True

# Wanna save the polarisation figure?
save_fig = True  # save results to fig (default = png)

# if auto_freq, frequency will be chosen automatically around f0
# if false, give the range of the frequency
auto_freq = False
limfreq_min = 0.1
limfreq_max = 0.4

max_amplitude = 10

# spacing of the ticks on the frequency distribution
steps = 0.1

#######################
HV_file = os.path.join(data_folder, '8N_%s.hv.grid'%ID)
print(HV_file)
out_folder = data_folder
# plot it
A_max, max_freq, max_Azi,A_min, min_freq, min_Azi = plot_polarisation_data(HV_file, ID, limfreq_min, limfreq_max, max_amplitude)